In [1]:
### 5.误差反向传播
import numpy as np
import sys
import matplotlib.pyplot as plt

In [2]:
###随机梯度SGD
class SGD:
    def __init__(self, lr=0.01):
        self.lr = lr
    def update(self,param,grad):
        if isinstance(param,dict):
            for key in param:
                param[key] -= self.lr * grad[key]
        else:
            param -= self.lr*grad
        return param

In [3]:
##动量法
class Momentum:
    def __init__(self, lr=0.01, momentum=0.9):
        self.lr = lr
        self.momentum = momentum
        self.v=None
    def update(self,param,grad):
        if self.v is None:
            self.v=dict()
            for i in param.keys():            
                self.v[i]=np.zeros_like(param[i])
        for key in param.keys():
            self.v[key] = self.momentum*self.v-self.lr * grad[key]
            param[key] += self.v[key]

        
        

In [ ]:
class AdaGrad:
    def __init__(self,lr=0.01):
        self.lr = lr
        self.h = None